In [1]:
import cv2
import numpy as np
from deepface import DeepFace

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
emotion_model = DeepFace.build_model('Emotion')

def face_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def mood_detection(image):
    faces = face_detection(image)
    emotions = []
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        detected_emotions = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
        emotion = max(detected_emotions[0]['emotion'].items(), key=lambda x: x[1])[0]
        emotions.append(emotion)
    return emotions

def my_face(image, face_folder):
    faces = face_detection(image)
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        detected_faces = DeepFace.find(face, db_path=face_folder, enforce_detection=False)
        if len(detected_faces) > 0:
            return True
    return False

# Test function
def test(face_folder):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if my_face(frame, face_folder):
            mood = mood_detection(frame)
            cv2.putText(frame, f"Regala's emotion is: {mood}", (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Not Regala", (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('test', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [2]:
face_folder = r'C:\Users\ThreeThree\Desktop\compiledcodes\datasci\Regala\Regala'
test(face_folder)

24-03-25 15:09:50 - Searching [[[ 52  71  46]
  [ 52  73  48]
  [ 55  76  52]
  ...
  [ 18  22  17]
  [ 17  21  16]
  [ 18  22  17]]

 [[ 56  76  50]
  [ 55  76  51]
  [ 56  77  53]
  ...
  [ 20  23  17]
  [ 20  23  17]
  [ 20  25  18]]

 [[ 61  81  55]
  [ 60  81  55]
  [ 60  81  56]
  ...
  [ 20  22  16]
  [ 19  22  16]
  [ 21  24  18]]

 ...

 [[ 54  77 112]
  [ 56  79 115]
  [ 59  80 118]
  ...
  [ 69  81 123]
  [ 70  83 123]
  [ 69  83 122]]

 [[ 49  72 108]
  [ 54  76 112]
  [ 59  79 117]
  ...
  [ 67  79 121]
  [ 70  82 123]
  [ 73  86 125]]

 [[ 52  70 110]
  [ 58  75 115]
  [ 62  78 118]
  ...
  [ 71  83 122]
  [ 71  84 123]
  [ 70  83 123]]] in 292 length datastore
24-03-25 15:09:50 - find function duration 1.258284568786621 seconds
24-03-25 15:09:55 - Searching [[[ 64  81  51]
  [ 68  83  52]
  [ 71  85  54]
  ...
  [ 29  27  27]
  [ 29  27  26]
  [ 31  30  27]]

 [[ 71  89  56]
  [ 75  91  58]
  [ 79  93  61]
  ...
  [ 28  26  25]
  [ 29  28  25]
  [ 31  30  26]]

 [[ 71  8